In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return w1, keep, layer3, layer4, layer7
tests.test_load_vgg(load_vgg, tf)

def tf_norm(sd=0.01):
    ''' Return Truncated normal initializer. Helper func '''
    return tf.truncated_normal_initializer(stddev=sd)


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    
    reg = tf.contrib.layers.l2_regularizer(1e-3)
    

    L7      = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1, kernel_initializer=tf_norm(sd=0.01),kernel_regularizer=reg)

    L7_up   = tf.layers.conv2d_transpose(L7, num_classes, 
                                kernel_size=4, strides=2, padding='same',
                                kernel_initializer=tf_norm(sd=0.01),
                                kernel_regularizer=reg)
    
    L4      = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1, kernel_initializer=tf_norm(sd=0.01),kernel_regularizer=reg)

    L4_L7    = tf.add(L4, L7_up)

    L4_L7_up   = tf.layers.conv2d_transpose(L4_L7, num_classes,
                                kernel_size=4, strides=2, padding='same',
                                kernel_initializer=tf_norm(sd=0.01),
                                kernel_regularizer=reg)
    
    L3      = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1, kernel_initializer=tf_norm(sd=0.01),kernel_regularizer=reg)

    L3_L4_L7   = tf.add(L4_L7_up, L3)  

    L3_L4_L7_up   = tf.layers.conv2d_transpose(L3_L4_L7, num_classes,
                                kernel_size=16, strides=8, padding='same',
                                kernel_initializer=tf_norm(sd=0.01),
                                kernel_regularizer=reg)

    return L3_L4_L7_up
    
    
#    return output
tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    logits          = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    correct_label   = tf.reshape(correct_label, (-1, num_classes), name='correct_label')
    
    #logits = tf.nn.sigmoid(nn_last_layer)
    #loss = tf.losses.sigmoid_cross_entropy(correct_label,logits)
    l2_loss = sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label)+l2_loss)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    
    return logits, train_op, loss
tests.test_optimize(optimize)


def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    tf.global_variables_initializer().run()
    tf.tables_initializer().run()
    KEEP_PROB = 0.5
    LEARNING_RATE = 0.001
    # TODO: Implement function
    for i in range(epochs):
        for image, label in get_batches_fn(batch_size):
            feed_dict = {input_image: image,
                        correct_label: label,
                        keep_prob: KEEP_PROB,
                        learning_rate: LEARNING_RATE} ##TODO: is the learning_rate reduced with adagrad?
            _,loss = sess.run([train_op, cross_entropy_loss],feed_dict)
            print ("epoch: "+str(i)+ " loss: "+str(loss))
    return 
tests.test_train_nn(train_nn)


def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    
    epochs = 2000
    batch_size=16
    

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        layer_output = layers(layer3_out, layer4_out, layer7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(layer_output,correct_label,learning_rate,num_classes)
        
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, image_input,
             correct_label, keep_prob, learning_rate)

        model_dir = './models'
        image_shape = (160, 576)

        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)


        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()

/home/josealb/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow Version: 1.6.0
Default GPU Device: /device:GPU:0
Tests Passed
Tests Passed
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


/home/josealb/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/home/josealb/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

with tf.Session() as sess:
    model_filename ='saved_model.pb'
    with gfile.FastGFile(model_filename, 'rb') as f:
        data = compat.as_bytes(f.read())
        sm = saved_model_pb2.SavedModel()
        sm.ParseFromString(data)
        g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)

LOGDIR='.'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)